In [2]:
import os
import sys

# configuring to  access the  AIMA repo and access some extras
if sys.platform == "win32":
    module_path = os.path.abspath(os.path.join('..\..'))
    ExtraFolderPath = os.path.abspath(os.path.join('..\EXTRA_DIR'))
else:
    module_path = os.path.abspath(os.path.join('../..'))
    ExtraFolderPath = os.path.abspath(os.path.join('../EXTRA_DIR'))
    
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(ExtraFolderPath)
sys.path.insert(1, module_path) 
sys.path.insert(2, module_path)

In [3]:
from agents import *
from notebook import psource


In [4]:
'''
In this context what we examine is the taxi driver world.Here the taxi driver is supposed to be driving in pre determined pattern {zig zag} , find a customer and drop him to the desired location.
There are some more variables such as fuel levels and cab occupancy.

The problem can be formulated as follows:

• Initial state: Any state can be designated as the initial state however i have assumed it to be [0,0] 
• Actions: In this simple environment, each state has just following  actions: turn Left or  Right if a bump has been detected, move forward , pickup and drop a customer and fill petrol.
• Goal test: This whether all the existing customers have been serviced given adequate fuel levels. 
• Path cost: Each step costs 5 units of fuel. The agent would die if it runs out of fuel. 

''' 

'\nIn this context what we examine is the taxi driver world.Here the taxi driver is supposed to be driving in pre determined pattern {zig zag} , find a customer and drop him to the desired location.\nThere are some more variables such as fuel levels and cab occupancy.\n\nThe problem can be formulated as follows:\n\n• Initial state: Any state can be designated as the initial state however i have assumed it to be [0,0] \n• Actions: In this simple environment, each state has just following  actions: turn Left or  Right if a bump has been detected, move forward , pickup and drop a customer and fill petrol.\n• Goal test: This whether all the existing customers have been serviced given adequate fuel levels. \n• Path cost: Each step costs 5 units of fuel. The agent would die if it runs out of fuel. \n\n'

In [5]:
class TheDumbTaxiDriver(Agent):
    current_cust_drop_loc=[2,2]
    is_customer_in=False
    iteration=0
    location=[0,1]
    direction=Direction('right')
    fuel_level=80
    def drive(self,thing):
        if  isinstance(thing,Customer):
            print('The agent is driving')
            return True
        else:
            False
    def hasEnoughFuel(self):
        if (self.fuel_level==0):
            return False
        else:
            return True
        
    def consumeFuel(self):
        self.fuel_level-=2
        return True
    
    def fillup(self,thing):
        if isinstance(thing,Petrol) and (self.fuel_level<50):
        
            print('Petrol low!!The agent is filling up gas..fuel level:{}'.format(self.fuel_level))
            self.fuel_level+=30
            return True
        else:
            print('Petrol enough!!The agent decided not to fill up gas..fuel_level:{}'.format(self.fuel_level))
            return False
        
        return False
    
    ''''def movedown(self):
        self.location += 1''' 
    
    def turn(self,d):
        print('Inside turn..turning {}'.format(d))
        self.direction= self.direction+d
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        self.consumeFuel()
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
        
        
        


In [6]:
class Customer(Thing):
    def __init__(self,drop_location=[1,1]):
        self.drop_location=drop_location
    
class Petrol(Thing):
    pass
class Bump(Thing):
    pass

    

class City(GraphicEnvironment):
    def percept(self, agent):
        things=self.list_things_at(agent.location)
        
        
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        if not self.is_inbounds(loc):
            things.append(Bump())
            
        return things

    def execute_action(self, agent, action):
        print('Inside Execute Action for Iteration:{0}  and action:{1} with direction:{2} and fuel level:{3}'.format(agent.iteration,action,agent.direction.direction,agent.fuel_level) )
        agent.iteration += 1
        if(agent.hasEnoughFuel()):
        
        
        
        
            if (action == 'turn'  and agent.direction.direction=='right'):

                agent.location[1] += 1
                agent.direction.direction ='left'
                agent.consumeFuel()
                print('turn and Right at  location:{0}  and action:{1} with direction:{2}'.format(agent.location,action,agent.direction.direction) )

            elif (action == 'turn'  and agent.direction.direction=='left' ):

                agent.location[1] += 1
                agent.direction.direction ='right'
                agent.consumeFuel()
                print('turn and Left at  location:{0}  and action:{1} with direction:{2}'.format(agent.location,action,agent.direction.direction) )

            elif action=='moveforward':

                if agent.direction.direction == Direction.R:
                    agent.location[0] += 1

                elif agent.direction.direction == Direction.L:
                    agent.location[0] -= 1
                agent.consumeFuel()
                


            if action=='pickup':
                    print('Action is to pick  up a customer..')
                    things=self.list_things_at(agent.location,tclass=Customer)
                    if agent.is_customer_in==False:
                        if len(things)!= 0:
                            agent.is_customer_in=True
                            thing = things[0]
                            agent.current_cust_drop_loc=thing.drop_location
                            print('Driver found a customer at location  {} and boarded him '.format(agent.location))
                            agent.drive(thing)                   
                            self.delete_thing(things[0])
                    else:
                        self.delete_thing(things[0])
                        print('Agent decided not to board the new customer coz there is an existing customer')
                        agent.moveforward()
                    agent.consumeFuel()
            if action == 'fillup':
                    things=self.list_things_at(agent.location,tclass=Petrol)
                    if len(things)!= 0:
                        agent.fillup(things[0])
                        print('Agent found a gas station at location:{0} and petrol level:{1}'.format(agent.location,agent.fuel_level))
                        self.delete_thing(things[0])

            if agent.location==agent.current_cust_drop_loc and agent.is_customer_in==True:
                print('dropping off the current customer to the drop location')
                agent.is_customer_in=False                
        else:
            agent.alive=False            
            print('Agent has run out of gas.Agent is dead.')               

        return action
    
    
    def is_done(self):
     
        no_edibles = not any(isinstance(thing, Customer) or isinstance(thing, Petrol) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        if(no_edibles):
            print('No more work  today..Let"s rest')
        return no_edibles or dead_agents
    

In [7]:
def program(percepts):
    
    for percept in percepts:
        
        if isinstance(percept,Customer):
            
            return 'pickup'
        elif isinstance(percept,Petrol):
            return 'fillup'
        elif isinstance(percept,Bump):
            
            return 'turn'

    
    return 'moveforward'


        
    

In [22]:

import random
size_of_x=10
size_of_y=3
a=random.randint(0,size_of_x-1)
b=random.randint(0,size_of_y-1)
print(a,b)
city = City(size_of_x,size_of_y, color={'TheDumbTaxiDriver': (200,0,0), 'Customer': (0, 200, 200), 'Petrol': (230, 115, 40)}) # park width is set to 5, and height to 20
theDumbTaxiDriver = TheDumbTaxiDriver(program)
customer = Customer(drop_location=[2,1])
customer2 = Customer(drop_location=[2,2])
customer3 = Customer(drop_location=[5,2])
petrol = Petrol()
more_petrol=Petrol()
city.add_thing(theDumbTaxiDriver,[0,0])
city.add_thing(customer, [random.randint(0,size_of_x-1),random.randint(0,size_of_y-1)])
city.add_thing(customer2,[random.randint(0,size_of_x-1),random.randint(0,size_of_y-1)])
city.add_thing(customer3,[random.randint(0,size_of_x-1),random.randint(0,size_of_y-1)])
city.add_thing(petrol,   [random.randint(0,size_of_x-1),random.randint(0,size_of_y-1)])
#city.add_thing(more_petrol,[1,0])

city.run(28)



7 1


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Inside Execute Action for Iteration:0  and action:pickup with direction:right and fuel level:80
Action is to pick  up a customer..
Driver found a customer at location  [0, 0] and boarded him 
The agent is driving


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Inside Execute Action for Iteration:1  and action:moveforward with direction:right and fuel level:78


...

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Inside Execute Action for Iteration:26  and action:moveforward with direction:right and fuel level:26


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Inside Execute Action for Iteration:27  and action:moveforward with direction:right and fuel level:24


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


In [18]:
import random
size_of_x=10
size_of_y=3
a=random.randint(0,size_of_x-1)
b=random.randint(0,size_of_y-1)
print(a,b)

9 2
